In [40]:
import sys
import os.path
sys.path.append("../classes/")

In [41]:
import pandas as pd 
import numpy as np
from math import log2
from Node import * 
from ImpurityMeasure import * 
from PandasToNumpy import PandasToNumpy
from ModelSelection import * 
from Metrics import * 
import operator 

# Get the cleaned csv file
Data cleaning is done by<i> clean_data.ipynb</i>

In [42]:
NB_DIR = %pwd

In [43]:
PATH = (f'{NB_DIR}/').replace('nbs/','')+'csv/'
CLEANED_FILE_NAME = 'mushrooms_cleaned.csv'

In [44]:
df = pd.read_csv(PATH+CLEANED_FILE_NAME)
df.shape

(5644, 23)

In [45]:
#Shuffle the dataset
df = df.sample(frac=1)

In [46]:
df.shape

(5644, 23)

In [47]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
3294,e,f,f,e,t,n,f,c,b,u,...,s,g,w,p,w,o,p,k,v,d
121,e,x,y,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,n,n,m
4355,p,f,y,g,f,f,f,c,b,g,...,k,p,p,p,w,o,l,h,v,p
4240,p,x,s,b,t,f,f,c,b,p,...,s,w,w,p,w,o,p,h,s,g
3173,p,x,f,g,f,f,f,c,b,p,...,k,n,n,p,w,o,l,h,v,p


In [48]:
y = df['class']

In [49]:
X = df.drop(['class'], axis=1)

In [50]:
X,y = PandasToNumpy.dataframe_to_numpy(X,y)

In [51]:
X,y = X[1:], y[1:] #drop the header 

# Create training and test set

In [13]:
X_train, X_test, y_train, y_test = ModelSelection.train_test_split(X,y)

In [14]:
def count_values(arr_list): 
    '''
    Find the number of unique values in the the array and return a dictionary with the unique values as key and the total number of them as value
    '''
    result = {} 
    for value in arr_list: 
        if value not in result: result[value] = 1
        else: result[value]+=1
    return result

In [15]:
def case_prediction(x,node): #x = [0,0,1,1]
    '''
    Arguments:
        x: 1D array. Example: values for an observation in the mushroom dataset [0,1,0,1....](23 columns)
        node: Node objekt
    
    If it is miss classified it will return None, otherwise it will return 0 or 1
    '''
    while len(x) > 0: 
        cat_var = x[node.category] #Rot sin kategori, så da velges posisjon 0 for eksempel 
        x = np.delete(x, node.category, axis=0)
        if cat_var not in node.children: #miss classification  
            node.data
            return None
        child_node = node.children[cat_var]
        if child_node.isLeaf: 
            return child_node.data
        node = child_node

In [16]:
def predict(X, node):
    '''
    Arguments:
        X: 2D array. Example: Alle the observations that you want to predict on the mushroom dataset
        node: Node object
    
    return an 1D array with all the predictions
    '''
    result = []
    for i in X: 
        result.append(case_prediction(i,node))
    return np.array(result)

In [17]:
def find_best_split(categories, y, impurity_measure): 
    '''
    Arguments:
        categories: training data with the remaining data. Each of the rows are a category. 
        y: true values for the training data 
        impurity_measure: String value if it's 'entropy' or 'gini'

    Return the information gain with highest value (The best category to select) 
    '''
    impurity_measure_method = getattr(ImpurityMeasure(), impurity_measure)
    
    impurity_measure_src = impurity_measure_method(y)
    best_ig_score = []
    for cat in categories:   #Example: cat can be cap-shape column.#cap-shape contains the variables: f, b, x 
      
        variables_dic = {}
        impurity_measure_of_branches = 0 if impurity_measure=='entropy' else impurity_measure_src
        
        #Variabel and y values for them
        for index,var in enumerate(cat): 
            if var not in variables_dic: variables_dic[var] = []
            variables_dic[var].append(y[index])
 
        #Calculate impurity measure for each of the variables
        for key, var_y in variables_dic.items(): 
            result = impurity_measure_method(var_y)*(len(var_y)/len(y))
            
            if(impurity_measure == 'entropy'):
                impurity_measure_of_branches += result
            else: 
                 impurity_measure_of_branches -= result
                
        information_gain = impurity_measure_src - impurity_measure_of_branches 
        best_ig_score.append(information_gain)
        
    return np.argmax(best_ig_score)

In [18]:
def find_dominant_value(values_dic): 
    '''
    value_dic: key is a binary number 0 or 1 and the value for them are number of them in a category. 
    return the key with the highest value in the dictionary
    '''
    return max(values_dic, key=values_dic.get)

In [19]:

def create_tree(X,y, node, impurity_measure): 
    '''
    Arguments:
        X: training data 
        y: true values for the training data 
        node: Node object 
        impurity_measure: String value if it's 'entropy' or 'gini'

    Creates the tree
    '''
    #Find best category 
    if len(X) == 0: 
        return 
    else: 
        index = find_best_split(X,y,impurity_measure) #Finding the index of the best category to split on
        
        node.category = index
        node.data = X[index] #add the category data to the node 
        child_dic = count_values(node.data) #count num variables for the category 
    
        for i in child_dic.keys(): #Add variable name and Node as child to the Node
            node.add_child(i,Node()) 
    
        for var, child_node in node.children.items():
            child_node.parent_node = node
            indices = [i for i, x in enumerate(node.data) if x == var] #the indices for the varible in the parent node data
            result = count_values(y[indices]) #check how many true and false values for the variable in the category
            if len(result) == 1: 
                child_node.data = next(iter(result.keys())) #The result is clean, so we give a value.
                child_node.isLeaf = True
            
            elif (len(X) == 1): #They are still not clean, but then you have to find the dominant value
                child_node.data = find_dominant_value(result)
                child_node.isLeaf = True
                
            if not child_node.isLeaf:
                node_X = np.delete(X, index, axis=0)
                create_tree(node_X[:,indices],y[indices], child_node, impurity_measure) 

In [20]:
def learn(X,y,impurity_measure = 'entropy',  pruning=False): 
    '''
    Arguments:
        X: training data 
        y: true values for the training data 
        impurity_measure: default impurity measure is 'entropy'. Alternatives: 'gini'
        pruning: default False

    If the user want to use pruning on the three, the training set will be divided into two subsets
    Then we will create the three and use the pruning set to prune the tree, otherwise it will just create a tree. 
    Returns the tree
    '''
    X_pruning, y_pruning = [],[]
    if pruning: 
        X, X_pruning, y, y_pruning =  ModelSelection.train_test_split(X,y) #get pruning set the same was as test set, but this time we split the training set in two subsets. 
        
   
    root = Node()

    #X.T so that each of the categories comes on a line
    create_tree(X.T,y, root,impurity_measure)
    if pruning: 
        while pruning: 
            pruning = False 
            pruning_pred = predict(X_pruning,root) #Vanlig predikt 
            pruning_accuracy = Metrics.accuracy(y_pruning, pruning_pred)

            for children in root.children.values():
                leaf_node  = find_leaf(children)
                change_made = prune(leaf_node, pruning_accuracy,X_pruning,y_pruning, root)
                if (not pruning) and change_made: pruning = change_made
            
    return root

In [21]:
def find_leaf(node):
    '''
    Arguments:
        node: Node object. Example a child node of root. 
    
    return the leaf node for the node. 
    '''
    if node.isLeaf: 
        return (node)
    else: 
        for key, child in node.children.items():
            return find_leaf(child)

In [22]:
def find_parent_variabel(parent_node,grand_parent_node):
    '''
    Find the variabel value in grandparent node, so we can change the variables node to another node. In our case parent_variable_in_grand_parent

    returns the variabel value. 
    '''
    for key, node in grand_parent_node.children.items(): 
        if node == parent_node: return key

In [23]:
def prune(leaf_node, pruning_accuracy, X_pruning, y_pruning, tree): 
    '''
    Arguments:
        leaf_node: leaf_node of a subtree
        pruning_accuracy: accurcy on the pruning set 
        X_pruning: pruning set
        y_pruning: true values for the pruning set
        tree: the root node 

    The method checks if the accurcy on the pruning set increases if we change the parent node of the leaf_node to leaf node with the output 0 and/or 1. 
    The parent node will be the new leaf if it imporves the accuracy and the children of the parent node will be removed from the three
    return variable says if there have been any changes in the three (True/False)
    '''
    prun_acc = pruning_accuracy
    changes = False 
    parent = leaf_node.parent_node
    
    child_values_to_check = []
    child_values_to_check.append(bool(leaf_node.data))
    if(len(parent.children) >1): 
        child_values_to_check.append(not leaf_node.data)
    
    for i in child_values_to_check: 
        grand_parent = parent.parent_node

        dummy_node = Node()
        dummy_node.category = parent.category
        dummy_node.isLeaf = True
        dummy_node.data = i
        dummy_node.parent_node = grand_parent

        if not (grand_parent == None): 
            parent_variable_in_grand_parent = find_parent_variabel(parent, grand_parent)
            grand_parent.children[parent_variable_in_grand_parent] = dummy_node
  
        #predict
        pred = predict(X_pruning, tree)
        pred_acc = Metrics.accuracy(y_pruning,pred)
    
        if (prun_acc < pred_acc): 
            prun_acc = pred_acc
            parent = dummy_node
            changes = True
        elif not (grand_parent == None ): 
            grand_parent.children[parent_variable_in_grand_parent] = parent
    return changes 

In [24]:
def print_tree(node):
    '''
    Print the tree with recursion 
    '''
    if node.isLeaf: 
        print("-----Leaf_value------")
        print(node.data)
        print()
    else: 
        for key, child in node.children.items():
            if not child.isLeaf:print(key, child.children.keys())
            print_tree(child)

In [25]:
#print("Root: "+ str(tree.children.keys()))
#print()
#print_tree(tree)

In [26]:
tree = learn(X_train,y_train, pruning=True)

In [27]:
y_pred = predict(X_test,tree)

In [28]:
Metrics.accuracy(y_test,y_pred)

1.0

# Performance measurement
Confusion matrix and recall and precison calculation with sklearn 

In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [30]:
#Change the None values in y_pred to a binary number.
#Return fixed y_pred 
def missclassification_binary(y_test,y_pred): 
    binary_y_pred = []
    for idx,i in enumerate(y_pred):
        if i == None: 
            i = not y_test[idx]
        binary_y_pred.append(i)
    return binary_y_pred

In [31]:
y_pred = missclassification_binary(y_test,y_pred)

In [32]:
confusion_matrix(y_test, y_pred)

array([[413,   0],
       [  0, 715]])

In [33]:
recall_score(y_test, y_pred)  

1.0

In [34]:
precision_score(y_test, y_pred)

1.0

# 4 discussion
I am shuffling the dataframe before I start to use it with <i> df = df.sample(frac=1)</i>
In addition I take out random rows for the test/pruning set. I am doing this to be sure that the data is randomly sampled.
Default value for test/pruning sets are 20% of the data set we give into the train_test_split method.


Entropy gives a better result than gini. The pruning has no effect on the tree when we are using entropy.
On gini it makes the result vary slightly.
The reason why pruning sometimes will reduce the size of the tree is that the sum of errors for leaf nodes (children) for a node is greater than the sum of error for that node. 

# Sklearn decsion tree

In [35]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

In [36]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [37]:
sk_predict = clf.predict(X_test)

In [38]:
Metrics.accuracy(y_test, sk_predict)

1.0

# 5 discussion
My implementation gives the same result as sklearn decsion tree with entropy as impurity measurement.
Both classifiers have a 100% accuracy on the test set(20% of the dataset).<br>
Pruning is something which is planned to be done in future in sklearn(https://datascience.stackexchange.com/questions/26087/sklearn-missing-pruning-for-decision-trees), so I can't compare that. 